In [ ]:
"""
pandas某些行的transformation
由于机器学习模型只接受数值，所以要将不同的数据类型转为数值

"""
import pandas as pd
import numpy as np

"""
字段说明
   1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
   2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
   3. month - month of the year: "jan" to "dec" 
   4. day - day of the week: "mon" to "sun"
   5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
   6. DMC - DMC index from the FWI system: 1.1 to 291.3 
   7. DC - DC index from the FWI system: 7.9 to 860.6 
   8. ISI - ISI index from the FWI system: 0.0 to 56.10
   9. temp - temperature in Celsius degrees: 2.2 to 33.30
   10. RH - relative humidity in %: 15.0 to 100
   11. wind - wind speed in km/h: 0.40 to 9.40 
   12. rain - outside rain in mm/m2 : 0.0 to 6.4 
   13. area - the burned area of the forest (in ha): 0.00 to 1090.84 
   (this output variable is very skewed towards 0.0, thus it may make
    sense to model with the logarithm transform). 
"""

df = pd.read_csv("./input/forestfires.csv", sep=",") 
df.head(5)


In [ ]:
df.info()

In [ ]:
# 分类属性转为数值数据 onehot
df_dummy = pd.get_dummies(df, columns=["month", "day"], prefix_sep="__")
df_dummy.head(5) 

In [ ]:
# dataframe转为ndarray
nparray = df_dummy.values
print(type(nparray))
nparray

In [ ]:
# 数值属性的各种标准化或者压缩

# 连续数据分箱 - 连续数据离散化的一种方法
self_define_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
df["FFMC_3_bin"] = pd.cut(df["FFMC"], bins=3)
df["FFMC_self_define_bin"] = pd.cut(df["FFMC"], bins=self_define_bins)
df["FFMC_label"] = pd.cut(df["FFMC"], bins=self_define_bins, labels=False)
df.head(5)

In [ ]:
# 字段的变化，创建一个新字段, 并且drop老的字段
# 也可以对分类属性做自己的编码，一般也是one-hot编码。
def reset_day(day):
    if day == "fri" or day == "sun" or day == "sat":
        return "second"
    else: 
        return "first"

df["part"] = df["day"].apply(reset_day)
print(df.head(10))
df.drop(["day"], axis=1, inplace=True)
print(df.head(10))

In [ ]:


# 构建测试的dataframe来测试流水线方法
df2 = pd.DataFrame({
    "city": ["tokyo", "tokyo", "london", "seattle", "san fransciso", "tokyo"],
    "boolean": ["yes", "no", "no", "no", "no", "yes"],
    "ordinal": ["somewhat like", "like", "somewhat like", "like", "somewhat like", "dislike"],
    "quantitative": [1, 11, -.5, 10, 8.3, 20]
}
)
df2
df2_copy = df2.copy()

In [ ]:
# 分类属性处理
df2 = pd.get_dummies(df2, columns=["city", "boolean"], prefix_sep="__")
df2

In [ ]:
# 定序属性 - 可以认为某些列的值比另外更好，实际运用中，是不是one hot编码也可以，尤其是DL
ordering = ["dislike", "somewhat like", "like"]
df2["ordinal"] = df2["ordinal"].map(lambda x: ordering.index(x))
df2

In [ ]:
# 数据分箱
df2["quantitative"] = pd.cut(df2["quantitative"], bins=3, labels=False)
df2

In [ ]:
df2_copy

In [ ]:
# pipeline方式来转换，更好的组织 
# 实际生产中应该采取这种方式
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

class MyDummy(TransformerMixin):
    def __init__(self, cols=None):
        self.cols = cols
    
    def transform(self, X):
        temp = X.copy()
        return pd.get_dummies(X, columns=self.cols)
    
    def fit(self, *_):
        return self
    
class MyOrdinal(TransformerMixin):
    def __init__(self, ordering=None):
        self.ordering = ordering
    
    def transform(self, X):
        temp = X.copy()
        temp["ordinal"] = X["ordinal"].map(lambda x: self.ordering.index(x))
        return temp
    
    def fit(self, *_):
        return self
    
class MyCut(TransformerMixin):
#     def __init__(self):
    
    def transform(self, X):
        temp = X.copy()
        temp["quantitative"] = pd.cut(X["quantitative"], bins=3, labels=False)
        return temp
    
    def fit(self, *_):
        return self
    
my_dummy = MyDummy(cols=["city", "boolean"])
my_ordinal = MyOrdinal(ordering=ordering)
my_cut = MyCut()

pipe = Pipeline([("dummify", my_dummy), ("ordinal", my_ordinal), ("cut", my_cut)])
df2_copy

In [ ]:
pipe.fit(df2_copy)
df3 = pipe.transform(df2_copy)
df3
# df2和df3是一样的结果 